# Toronto Neighborhoods

https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

## Imports

In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
!conda install -c conda-forge bs4 --yes # uncomment this line if you haven't completed the Foursquare API lab
!pip install lxml



from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

import json
from bs4 import BeautifulSoup
from pprint import pprint


print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-1.22.0               |     pyh9f0ad1d_0          63 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         395 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.50-py_0
  geopy              conda-forge/noarch::geopy-1.22.0-pyh9f0ad1d_0

The following packages will be UPDATED:

  ca-certificates                     2020.4.5

## Scraping Data

In [23]:
url = 'https://en.wikipedia.org/w/api.php?action=parse&prop=text&format=json&redirects&page=List%20of%20postal%20codes%20of%20Canada:_M&section=1'
result = requests.get(url).json()
html_data= result['parse']['text']['*']
df = pd.read_html(html_data)[0]
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Cleaning Data

### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [24]:
df.drop(df[df["Borough"]=="Not assigned"].index, axis=0, inplace=True) # Drop all other categories
df.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


### More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

Already done in the table

### If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [25]:
def rating_function(x):
    if  x[2] == "Not assigned":
        return x[1]
    else:
        return x[2]

df["Neighborhood"] = df.apply(rating_function,axis=1)

In [26]:
df1 = df[df['Neighborhood'] == "Not assigned"] # Making sure that theres no "not assigned" neighborhoods
df1

,Postal Code,Borough,Neighborhood


### In the last cell of your notebook, use the .shape method to print the number of rows of your dataframe.

In [27]:
df.shape

(103, 3)